# PTE Dependence on Reflector Specular Ratio
Authors: Luc Barrett, Loick Marion

In the calculation of specular vs diffuse ratios, its assumed it has a linear effect on PTE value (first order approximation). To investigate if this is a good assumption, this notebook will sweep specular ratios and calculate the PTE, and plot the result so it can be evaluated if a linear approximation is appropriate. If not more complex optimization methods may be needed to determine these ratios.

In [ ]:
from analysis.geometry_manager import geometry_manager
from analysis.run_manager import run_manager
import numpy as np
import matplotlib.pyplot as plt

## Metadata and Info

In [ ]:
experiment_name = "SiliconeFlippedSourceUpper"
num_particles = 1_000_000
seed = 1042
run_id = 1
visualize = True
plots = []

exclusion = []
label = "silicon-upper-8reflector"

exclusions = [f"reflector{i}" for i in exclusion]  # map to name

## Run Simulations

In [ ]:
ptes = []
ptes_err = []
speculars = np.linspace(0, 1, 30)
for specular in speculars:
    print(f"Experiment Name:       {experiment_name }")
    print(f"Number of particles:   {str(num_particles) }")
    print(f"Random seed:           {str(seed) }")
    print(f"Run ID:                {str(run_id) }")
    print(f"Visualize:             {str(visualize) }")
    print(f"Plots:                 {plots}")
    print(f"Excluded reflectors:   {exclusion}")

    gm = geometry_manager(
        experiment_name=experiment_name,
        run_id=run_id,
        visualize=visualize,
        exclude=exclusion,
    )

    # set ratios of specular vs diffuse
    gm.surf_manager.surfaces["silicon"].set("reflect_specular", specular)
    gm.surf_manager.surfaces["silicon"].set("reflect_diffuse", 1 - specular)

    rm = run_manager(
        geometry_manager=gm,
        experiment_name=experiment_name,
        random_seed=seed,
        num_particles=num_particles,
        run_id=run_id,
        plots=plots,
        write=False,
    )
    pte = rm.ana_man.photon_transmission_efficiency
    ptes.append(pte)
    ptes_err.append(rm.ana_man.pte_st_dev)
    run_id += 1
print("Done!")

In [ ]:
print(ptes)
print(ptes_err)

## Plot Results

In [ ]:
plt.figure(figsize=(15, 10))
plt.errorbar(speculars, ptes, ptes_err, capsize=5)
plt.xlabel("Specular Fraction")
plt.ylabel("PTE")
plt.title("PTE as Function of Specular Ratio")
plt.grid()
plt.show()